In [1]:
%config Completer.use_jedi = False
import sys
import warnings
warnings.filterwarnings('ignore')

import os
# os.environ["CUDA_VISIBLE_DEVICES"]="2"
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# import tensorflow as tf
import numpy as np
from scipy import ndimage
import matplotlib.pyplot as plt

# MEMORY = 16*1024

# gpus = tf.config.list_physical_devices('GPU')
# try:
#     tf.config.set_logical_device_configuration(gpus[0],
#                                               [tf.config.LogicalDeviceConfiguration(memory_limit=MEMORY)])
#     logical_gpus = tf.config.list_logical_devices('GPU')
#     print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
# except RuntimeError as e:        
#     # Memory growth must be set before GPUs have been initialized
#     print(e) 
    
import pathlib
import pickle
from pathlib import Path
sys.path.append(f"{Path.home()}/morpho_repo")
sys.path.append(f"{Path.home()}/morpho_repo/turing_codebase")
from turing.utils import *
# from turing.tf_utils import *
# import turing.pinns as tu
# from turing.loss_functions import *

import pandas as pd

In [2]:
from pde_solvers.cn import *
from local_utils import *
from turing.three_nodes_circuits import create_circuit_3954

# Load the parameters

In [3]:
df = pd.read_csv("../df_network_analysis.csv")
df["adj_tup"] = df["adj_tup"].apply(lambda x: eval(f"tuple({x})"))
df["Adj"] = df["adj_tup"].apply(lambda x: np.array(x).reshape((3,3)))

In [4]:
adj=np.array([[1, 1, -1], [-1, 0, -1], [0, -1, 1]])
subnet_list = [g[1] for g in df.groupby("adj_tup") if g[0] == tuple(adj.flatten())]
if len(subnet_list) == 0:
    print("================================")
    print("There is no adjacancy matrix as: ", adj)
    print("================================")
else:
    subnet_df = subnet_list[0]

In [5]:
subnet_df.head(1)

,min_A,min_B,min_C,avg_A,avg_B,avg_C,max_A,max_B,max_C,std_A,...,lb_CB,ub_CB,state_CC,lb_CC,ub_CC,Adj,k_max,params,path,adj_tup
116,0.879311,27.4671,2.00005,3.859942,31.707845,2.000238,6.759598,35.970982,2.000543,1.49011,...,1.780018,56.210562,Active,0.880009,28.110281,"[[1, 1, -1], [-1, 0, -1], [0, -1, 1]]",8.430177,[4.0e+00 1.0e-03 1.0e-01 5.0e+00 5.0e+00 5.0e+...,outputs_second_search/solution_10_0_24.npy,"(1, 1, -1, -1, 0, -1, 0, -1, 1)"


In [6]:
def load_dataset(path):
    with open(f"../{path}", "rb") as f:
        k_max, params, res = np.load(f, allow_pickle=True)
    (n_val, 
     b_A_val, mu_A_val, V_A_val, K_AA_val, K_AB_val, K_AC_val,
     b_B_val, mu_B_val, V_B_val, K_BA_val, K_BC_val,
     b_C_val, mu_C_val, V_C_val, K_CB_val, K_CC_val) = params
    params = {
              'D_A':0.01,
              'D_B':1.0,
              'n':n_val, 
              'b_A':b_A_val, 
              'mu_A':mu_A_val, 
              'V_A':V_A_val,
              'K_AA':K_AA_val, 
              'K_AB':K_AB_val,  
              'K_AC':K_AC_val,
              'b_B':b_B_val, 
              'mu_B':mu_B_val, 
              'V_B':V_B_val,
              'K_BA':K_BA_val, 
              'K_BC':K_BC_val,  
              'b_C':b_C_val, 
              'mu_C':mu_C_val, 
              'V_C':V_C_val,
              'K_CB':K_CB_val, 
              'K_CC':K_CC_val
             }
           
    return (params, res, k_max)

In [7]:
from scipy.optimize import minimize

def cos_dist(arr1, arr2):
    arr1_L = np.sqrt(np.dot(arr1, arr1))
    arr2_L = np.sqrt(np.dot(arr2, arr2))
    return np.dot(arr1, arr2)/(arr1_L*arr2_L)

def alienor_components2(epsilon, l1, bounds):
    """
    
    Args:
          epsilon (float): The accuracy of the estimates.
          l1 (float): Lipschitz constant
          bounds (list of tuples):
          bounds like [a_i, b_i], for each variables separatly.
          
          
    """
    n = len(bounds) 
    assert n >= 2, "The method expects two or more vairbales."
    assert np.all([len(item) == 2 
                   for item in bounds]), "bounds must be tuple of (a_i,b_i)"
    
    alpha = epsilon/(2*l1*np.sqrt(n-1))
    alphas = np.ones(n)
    
    def get_h_i(a,b,alpha):
        def h_i(t):
            return (a-b)*np.cos(alpha*t)/2 + (a+b)/2
        return h_i
    a,b = bounds[0]
    h_i = get_h_i(a,b,1)
    h_list = [h_i]
    for i in range(1, n):
        # [a_i, b_i]
        a,b = bounds[i]
        # alphas[i-1] (alpha/pi) / (|b_i| + |a_i|)
        alphas[i] = alpha*alphas[i-1]/(np.pi*(np.abs(b)+np.abs(a)))
        # h_i = (a_i - b_i)cos(alpha_i theta)/2 +  (a_i + b_i)/2         
        h_i = h_i = get_h_i(a,b,alphas[i])#
        h_list.append(h_i)        
        
        
    # l2 or Lipschitz constant of the aliemor h functions
    l2 = np.linalg.norm([(np.abs(d[1])+np.abs(d[0]))**2 * a**2 for d,a in zip(bounds,alphas) ])/2
    #
    theta_max = np.pi/alphas[-1]    
    return alpha, alphas, l2, h_list, theta_max

def minim_2(epsilon, l1, bounds, func, maxiter=10000):
    alpha, alphas, l2, h_list, theta_max = alienor_components2(epsilon, l1, bounds)
    k = 1
    L = l1 * l2
    theta = epsilon / L    
    theta_epsilon = theta
    
    f = lambda t: func(*[ h(t) for h in h_list])
    f_epsilon = f_theta = f(theta_epsilon)
    
    while k < maxiter:
        if theta > np.pi/alphas[-1]:            
            return k, theta,theta_epsilon, f_epsilon, ""
        
        theta = theta + (epsilon + f_theta - f_epsilon)/ L        
        f_theta  =  f(theta)
        if f_theta < f_epsilon:
            f_epsilon = f_theta
            theta_epsilon = theta
        k += 1    
    return k, theta,theta_epsilon, f_epsilon, f"max iteration '{maxiter}' is reached"

In [8]:
from multiprocessing import Pool, shared_memory

class _LocalFunctions:
    @classmethod
    def add_functions(cls, *args):
        for function in args:
            setattr(cls, function.__name__, function)
            function.__qualname__ = cls.__qualname__ + '.' + function.__name__

In [13]:
import copy

D_A_val, D_B_val = 0.01, 1.0
N=5000
T=100
delta_t = T/N
model_128_10 = RD_2D_1st_Order(Ds=[D_A_val, D_B_val, 0], 
                               delta_t=delta_t, Lx=10, Ly=10, 
                               Ix=128, Jy=128,
                               boundary_condition=Neumann_Boundary_2D)



In [14]:
pool_num = 28
models_list = np.array([ copy.deepcopy(model_128_10) for _ in range(pool_num)])

In [23]:
def to(arr):
    return arr.reshape(128, 128) 

def reshape(arr, steps=1):
    T = arr.shape[0]
    ret = np.array([
        [to(arr[i, 0, :]), to(arr[i, 1, :]), to(arr[i, 2, :])]
        for i in range(T-steps, T)
    ])
    return np.einsum("tcxy -> cxyt", ret)

def rmse(arr1, arr2):
    return np.sqrt(np.mean((arr1-arr2)**2))


def G(sigma, x, y):
    gaussian = (1/(2*np.pi*sigma**2))*np.exp(-(x**2+y**2)/(2*sigma**2))
    return gaussian

def G_discrete(sigma, n):
    l = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            l[i,j] = G(sigma, (i-(n-1)/2),(j-(n-1)/2))
    return l

def G_discrete_normalise(sigma, n):
    l = G_discrete(sigma, n)
    return l/np.sum(l)

def single_simulation(args):
    (iter_i, 
     (simulate_from_start,
     i,
     index, 
     run, 
     path, 
     (b_A_val, V_A_val,  K_AA_val,  K_BA_val), 
     (b_B_val, V_B_val,  K_AB_val,  K_CB_val), 
     (b_C_val, V_C_val,  K_AC_val,  K_BC_val, K_CC_val))) = args
    
    (params, res_1, _) = load_dataset(path)    
    if simulate_from_start:
        A_init = res_1[0, 0, :]
        B_init = res_1[0, 1, :]
        C_init = res_1[0, 2, :]
    else:
        A_init = res_1[-1, 0, :]
        B_init = res_1[-1, 1, :]
        C_init = res_1[-1, 2, :]
    
    n_val = params["n"]
    mu_A_val = params["mu_A"]
    mu_B_val = params["mu_B"]
    mu_C_val = params["mu_C"]
        
    actual_params = np.array(list(params.values())[3:])
    estimated_params = np.array(
        [b_A_val, mu_A_val, V_A_val, K_AA_val, K_AB_val,
         K_AC_val, b_B_val, mu_B_val, V_B_val, K_BA_val,
         K_BC_val, b_C_val, mu_C_val, V_C_val, K_CB_val, K_CC_val])
    euclidian_dist = np.linalg.norm(actual_params-estimated_params)
    c_dist = cos_dist(actual_params, estimated_params)
    kinetics = create_circuit_3954(n_val, 
                                   b_A_val, mu_A_val, V_A_val, K_AA_val, K_AB_val, K_AC_val,
                                   b_B_val, mu_B_val, V_B_val, K_BA_val, K_BC_val,
                                   b_C_val, mu_C_val, V_C_val, K_CB_val, K_CC_val)  
    model_shm = shared_memory.SharedMemory(name="models_list")
    models = np.ndarray((pool_num,), 
                        dtype=models_list.dtype, 
                        buffer=model_shm.buf)
    model_128_10 = models[iter_i%pool_num]

    res_2 = model_128_10.integrate([A_init,B_init,C_init], kinetics, 5000-1, 500)
        
    with open(f"./temp/res_{index}_{run}_{i}_{0 if simulate_from_start else 1}.npy", "wb") as f:
        np.save(f, res_2)
    
    return (simulate_from_start,
            i,
            index, 
            run, 
            path,
            (n_val, 
             b_A_val, mu_A_val, V_A_val, K_AA_val, K_AB_val, K_AC_val,
             b_B_val, mu_B_val, V_B_val, K_BA_val, K_BC_val,
             b_C_val, mu_C_val, V_C_val, K_CB_val, K_CC_val),
             euclidian_dist,
             c_dist)
    


In [49]:
# with open(f"./temp/temp_file_2.npy", "wb") as f:
#     np.save(f, results)
n_max = 20
with open(f"./temp/temp_file.npy", "rb") as f:
     results = np.load(f, allow_pickle=True)
with open(f"./temp/temp_file_2.npy", "rb") as f:
    results_second = np.load(f, allow_pickle=True)
results = np.concatenate([results, results_second])

In [33]:
simulate_from_start = True
args_list = [
    (simulate_from_start,
     i,
     dict_params["index"],
     dict_params["run"],
     dict_params["path"],
     dict_params["final_params_A"][i], 
     dict_params["final_params_B"][i], 
     dict_params["final_params_C"][i]) for dict_params in results
                                       for i in range(n_max) 
]

args_list = [ (iter_i, item)
    for iter_i, item in enumerate(args_list)]

In [34]:
shm = shared_memory.SharedMemory(name="models_list",
                                 create=True, 
                                 size=models_list.nbytes)
shared_models_list = np.ndarray((pool_num,), 
                                 dtype=models_list.dtype,
                                 buffer=shm.buf)

shared_models_list[:] = models_list[:]

In [35]:

with Pool(pool_num) as pool:
    results2 = pool.map(single_simulation, args_list)

In [36]:
model_shm = shared_memory.SharedMemory(name="models_list")
model_shm.close()
model_shm.unlink()

In [41]:
with open(f"./temp/temp_file2_3_from_start.npy", "wb") as f:
    np.save(f, results2)

In [56]:
subnet_df.iloc[23]

min_A                                                0.124198
min_B                                               25.394481
min_C                                               10.509365
avg_A                                                5.612208
avg_B                                               28.030212
avg_C                                                10.60795
max_A                                                8.326133
max_B                                               29.558658
max_C                                               10.821324
std_A                                                2.693908
std_B                                                0.880593
std_C                                                0.065706
state_AA                                               Active
lb_AA                                                0.880009
ub_AA                                               28.110281
state_AB                                               Active
lb_AB   

In [52]:
[ r for r in results if r['index'] == 23]

[{'index': 23,
  'run': 'no_noise',
  'path': 'outputs_second_search/solution_10_10_182.npy',
  'A_max': 8.598405696086237,
  'A_min': 0.04847241732848541,
  'B_max': 10.556187665164396,
  'B_min': 8.297796967985516,
  'C_max': 29.06259736161651,
  'C_min': 2.055726608531379,
  'final_loss_A': array([1040.32751465, 1126.57653809, 1213.6640625 , 1526.93725586,
         1625.08837891, 1683.02941895, 1792.00061035, 1917.26074219,
         1940.41589355, 1954.16931152, 1990.25158691, 2023.46557617,
         2029.16577148, 2130.90991211, 2302.84155273, 2673.8203125 ,
         2694.90600586, 2706.92285156, 2749.71118164, 2752.10961914]),
  'final_params_A': array([[3.21765114e-02, 1.31665016e+01, 4.67995323e+00, 1.25229894e+01],
         [3.84654296e-02, 1.76407811e+01, 5.16952739e+00, 1.08078342e+01],
         [1.76525509e-03, 1.12572122e+01, 4.70765842e+00, 1.54376151e+01],
         [6.89631869e-02, 9.75229229e+00, 5.05315276e+00, 2.71017941e+01],
         [2.55894714e-02, 1.00439553e+01, 

In [53]:
(params, res_1, _) = load_dataset('outputs_second_search/solution_10_10_182.npy')    

In [54]:
params

{'D_A': 0.01,
 'D_B': 1.0,
 'n': 4.0,
 'b_A': 0.001,
 'mu_A': 1.0,
 'V_A': 50.0,
 'K_AA': 5.0,
 'K_AB': 5.0,
 'K_AC': 25.0,
 'b_B': 0.1,
 'mu_B': 1.0,
 'V_B': 50.0,
 'K_BA': 20.0,
 'K_BC': 25.0,
 'b_C': 10.0,
 'mu_C': 1.0,
 'V_C': 50.0,
 'K_CB': 25.0,
 'K_CC': 25.0}